In [ ]:
import nltk
import re
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.

  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [ ]:
# 1.1
data = pd.read_csv('name_rus.txt', encoding='cp1251', header=None, names=['surname'])
print(data.head())


class Vocab():
    def __init__(self, _data):
        self.max_surname_len = _data.surname.str.len().max()
        self.token_to_id = {}
        self.id_to_token = {}
        self.tech = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']
        self.build_vocab(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя'))
        self.vocab_size = len(self.token_to_id)

    def build_vocab(self, letters):
        self.token_to_id = {token: idx for idx, token in enumerate(letters + self.tech)}
        self.id_to_token = {idx: token for token, idx in self.token_to_id.items()}

    def __len__(self):
        return self.vocab_size

    def __getitem__(self, token):
        return self.token_to_id[token]

    def __contains__(self, token):
        return token in self.token_to_id

    def to_tokens(self, ids):
        # объединим id в одну строку
        ids_sub = []
        for idx in ids:
            if self.id_to_token[int(idx)] not in self.tech:
                ids_sub.append(self.id_to_token[int(idx)])

        return ''.join(ids_sub)

    def to_ids(self, tokens):
        out = [self.token_to_id['<SOS>']] + [self.token_to_id['<PAD>']] * self.max_surname_len
        for i, token in enumerate(tokens, 1):
            if token not in self.token_to_id:
                out[i] = self.token_to_id['<UNK>']
            else:
                out[i] = self.token_to_id[token]
        out.append(self.token_to_id['<EOS>'])
        return out


vocab = Vocab(data)
np.random.seed(21 * 3)

test_tokens = np.random.randint(0, 32, 6)
print('\n', test_tokens, sep='')
print(vocab.to_tokens(test_tokens))
print(vocab.to_ids(vocab.to_tokens(test_tokens)))
print(vocab.to_tokens((vocab.to_ids(vocab.to_tokens(test_tokens)))))


      surname
0     авдокея
1     авдоким
2      авдоня
3    авдотька
4  авдотьюшка

[12 20 11 23 18  0]
лукцса
[34, 12, 20, 11, 23, 18, 0, 33, 33, 33, 33, 33, 33, 33, 35]
лукцса


In [ ]:
test_tokens

array([12, 20, 11, 23, 18,  0])

In [ ]:
class SurnameDataset(Dataset):

    def __init__(self, _data, _vocab):
        self.data = _data
        self.vocab = _vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        _x = self.vocab.to_ids(self.data.surname.iloc[idx])
        return torch.tensor(_x[:-1]), torch.tensor(_x[1:])


dataset = SurnameDataset(data, vocab)
x, y = next(iter(dataset))
print(x, y)
print(vocab.to_tokens(x))
print(vocab.to_tokens(y))


tensor([34,  0,  2,  4, 15, 11,  5, 32, 33, 33, 33, 33, 33, 33]) tensor([ 0,  2,  4, 15, 11,  5, 32, 33, 33, 33, 33, 33, 33, 35])
авдокея
авдокея


In [ ]:
class MyModelRNN(torch.nn.Module):

    def __init__(self, _vocab: Vocab, embedding_size, hidden_size):
        super(MyModelRNN, self).__init__()
        self.embedding = torch.nn.Embedding(_vocab.vocab_size, embedding_size)
        self.rnn = torch.nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, 1024)
        self.fc2 = torch.nn.Linear(1024, _vocab.vocab_size)
        self.f = torch.nn.Sigmoid()
        self.dropout = torch.nn.Dropout(0.25)
        self.vocab = _vocab

    def forward(self, x, h=None):
        x = self.embedding(x)
        x, h = self.rnn(x, h)
        x = self.dropout(self.fc(x))
        x = self.fc2(self.f(x))
        return x, h

In [ ]:
model = MyModelRNN(vocab, 256, 128)
print(model(torch.tensor([vocab.to_ids(name) for name in ['виктор', 'егор']])))
model(torch.tensor([vocab.to_ids(name) for name in ['виктор', 'егор']]))[0].shape

(tensor([[[ 0.1109,  0.7914, -0.0430,  ...,  0.0969,  0.6286,  0.1847],
         [ 0.0342,  0.7987,  0.0149,  ...,  0.0975,  0.5195,  0.1857],
         [ 0.0882,  0.8167,  0.0483,  ...,  0.0119,  0.5667,  0.1631],
         ...,
         [ 0.0538,  0.7993, -0.0260,  ..., -0.0667,  0.5354,  0.1516],
         [ 0.0469,  0.7862, -0.0269,  ...,  0.0017,  0.5860,  0.1439],
         [ 0.1976,  0.8402, -0.0628,  ...,  0.0577,  0.4790,  0.2603]],

        [[ 0.0653,  0.7593, -0.0293,  ...,  0.1302,  0.5845,  0.1603],
         [ 0.0557,  0.7981,  0.1300,  ..., -0.0087,  0.6054,  0.0569],
         [ 0.1592,  0.7599, -0.0496,  ...,  0.0597,  0.5202,  0.1930],
         ...,
         [ 0.0810,  0.7915, -0.0622,  ..., -0.0100,  0.5935,  0.1392],
         [ 0.0582,  0.8468, -0.0331,  ..., -0.0359,  0.5382,  0.1603],
         [ 0.1395,  0.8935, -0.0528,  ...,  0.0451,  0.5273,  0.1830]]],
       grad_fn=<ViewBackward0>), tensor([[[-0.8207,  0.5053,  0.9007,  0.1456, -0.8592,  0.2377, -0.5917,
         

torch.Size([2, 15, 37])

In [ ]:
def sample_next(model, x, prev_state, topk=5, uniform=True):
    out, state = model(x, prev_state)
    last_out = out[0, -1, :]
    topk = topk if topk else last_out.shape[0]
    top_logit, top_ix = torch.topk(last_out, k=topk, dim=-1)
    p = None if uniform else torch.nn.functional.softmax(top_logit.detach(), dim=-1).numpy()
    sampled_ix = np.random.choice(top_ix, p=p)
    return sampled_ix, state


def sample(model, start_letters, topk=5, uniform=False, max_seqlen=15, stop_on=None):
    model.eval()
    with torch.no_grad():
        sampled_ix_list = start_letters[:]
        x = torch.tensor([start_letters])

        prev_state = None
        for t in range(max_seqlen - len(start_letters)):
            sampled_ix, prev_state = sample_next(model, x, prev_state, topk, uniform)

            sampled_ix_list.append(sampled_ix)
            x = torch.tensor([[sampled_ix]])

            if sampled_ix == stop_on:
                break

    model.train()
    return sampled_ix_list


vocab.to_tokens(sample(model, [2], stop_on=vocab.token_to_id['<EOS>']))

'в'

In [ ]:
px.line(pd.DataFrame({'train loss': loss_log}))

In [ ]:
def train(_model: torch.nn.Module, epochs=100):
    optimizer = torch.optim.Adam(_model.parameters())
    loss = torch.nn.CrossEntropyLoss()
    loss_log = []
    loader = DataLoader(dataset, batch_size=512)
    for i in range(epochs):
        epoch_loss = 0
        j = 1  # Делители running losses
        _model.train()
        for j, (batch_x, batch_y) in enumerate(loader):
            y_pred = _model(batch_x)
            running_loss = loss(y_pred[0].reshape(-1, vocab.vocab_size), batch_y.reshape(-1))
            epoch_loss += running_loss.item()

            running_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        _model.eval()
        epoch_loss /= j

        if i % 25 == 0:
            print(f'EPOCH: {i + 1:3d} \t LOSS: {epoch_loss:0.4f}')

            eos = vocab.token_to_id['<EOS>']
            start = vocab.to_ids('викт')[1:5]
            samples = [vocab.to_tokens(sample(model, start, stop_on=eos)) for _ in range(3)]
            print('Викт ---> ', *samples, sep=' | ')

        loss_log.append(epoch_loss)

    return _model, loss_log


model, loss_log = train(model, epochs=76)

EPOCH:   1 	 LOSS: 3.9016
Викт --->  | виктаа | виктаа | виктаи
EPOCH:  26 	 LOSS: 1.5646
Викт --->  | виктита | викта | викта
EPOCH:  51 	 LOSS: 1.3830
Викт --->  | виктава | виктина | виктя
EPOCH:  76 	 LOSS: 1.3661
Викт --->  | виктья | виктья | виктичнич


In [ ]:
eos = vocab.token_to_id['<EOS>']
start = vocab.to_ids('викт')[1:5]
samples = [vocab.to_tokens(sample(model, start, stop_on=eos)) for _ in range(10)]
print('Викт ---> ', *samples, sep=' | ')

Викт --->  | виктия | викта | викта | викта | виктич | викта | виктия | викта | викта | викта


## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [ ]:
with open('The-Great-Gatsby.txt') as file:
    text = file.read()

text = nltk.word_tokenize(re.sub(r'[^A-Z]', '', text.lower(), -1), 'english')
text[:3]

['chapter', 'i', 'in']

In [ ]:
from tqdm import tqdm


class TextDataset(Dataset):

    def __init__(self, text, seq_len):
        self.text = text
        self.seq_len = seq_len
        self.tokens = list(set(text)) + [' ', '']
        self.token_to_id = {token: idx for idx, token in enumerate(self.tokens)}
        self.id_to_token = {idx: token for idx, token in enumerate(self.tokens)}
        self.num_tokens = len(self.tokens)

    def __len__(self):
        return len(self.text) // self.seq_len

    def __getitem__(self, idx):
        start_idx = idx * self.seq_len
        end_idx = start_idx + self.seq_len + 1
        text_str = self.text[start_idx:end_idx]
        text_encoded = [self.token_to_id[token] for token in text_str]
        x = torch.tensor(text_encoded[:-1])
        y = torch.tensor(text_encoded[1:])
        return x, y

    def decode(self, text_encoded):
        out = ''
        for idx in text_encoded:
            if int(idx) in self.id_to_token.keys():
                out += self.id_to_token[int(idx)]

            out += ' '

        return out

    def encode(self, text):
        out = []
        for token in text:
            if token in self.token_to_id:
                out.append(self.token_to_id[token])
            else:
                out.append(self.token_to_id[' '])
        return torch.tensor(out)

    def collate_fn(self, batch):
        x = torch.stack([item[0] for item in batch])
        y = torch.stack([item[1] for item in batch])
        return x, y

    def generate_batch(self, batch_size, seq_len, device='cpu'):
        while True:
            x = torch.randint(self.num_tokens, (batch_size, seq_len), dtype=torch.long, device=device)
            y = torch.randint(self.num_tokens, (batch_size, seq_len), dtype=torch.long, device=device)
            yield x, y


class RNN(torch.nn.Module):

    def __init__(self, num_tokens, emb_size, rnn_num_units, num_layers=1, dropout=0.5):
        super().__init__()
        self.emb = torch.nn.Embedding(num_tokens, emb_size)
        self.rnn = torch.nn.LSTM(emb_size, rnn_num_units, num_layers=num_layers, dropout=dropout)
        self.hid_to_logits = torch.nn.Linear(rnn_num_units, num_tokens)

    def forward(self, x):
        x = self.emb(x)
        x, _ = self.rnn(x)
        x = self.hid_to_logits(x)
        return x


def generate_sample(model, dataset, prime_str=' ', sample_len=100):
    model.eval()
    with torch.no_grad():
        x = dataset.encode(prime_str)
        x = x[None, :].to(next(model.parameters()).device)
        for _ in range(sample_len):
            logits = model(x)
            p_next = torch.nn.functional.softmax(logits[:, -1], dim=-1)
            next_token = torch.multinomial(p_next, num_samples=1)
            x = torch.cat([x, next_token], dim=1)
        return dataset.decode(x[0].cpu())


def train(model, dataset, num_epochs, batch_size, lr=1e-3, grad_clip=5, device='cpu'):

    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    loader = DataLoader(dataset, batch_size=batch_size)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for x, y in tqdm(loader, leave=False):
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits.transpose(1, 2), y)
            loss.backward()
            epoch_loss += loss.item()
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

        if epoch % 5 == 0:
            print(f'Epoch: {epoch + 1}, Loss: {epoch_loss:.4f}')
            print(generate_sample(model, dataset, sample_len=dataset.seq_len))


dataset = TextDataset(text, seq_len=15)
model = RNN(num_tokens=dataset.num_tokens, emb_size=256, rnn_num_units=256, num_layers=3, dropout=0.25)
train(model, dataset, num_epochs=50, batch_size=512)


Epoch: 1, Loss: 68.0098
  bound wailing lurches carelessness all-night opportunity. precisely confirmation. warm wallet sturdy swaying alleys pearl privy 


Epoch: 6, Loss: 50.0559
  torpedoes dressing-gowns coast packing acre tangle staring forgot he affair stoop pitiful contrast regal shouldered 


Epoch: 11, Loss: 49.9642
  wholesale chiefly remarked annoy montenegro massage life.myrtle hadn certainly ballooned tumultuous four intermittent hors-d threadbare 


Epoch: 16, Loss: 49.9583
  innocently depression. be drowsiness satisfactory sulkily quick lie. blue san beaver constantly meet expressive mile 


Epoch: 21, Loss: 49.9524
  brewer gravely dress—and contemptuously black weekbe last haven. ’ thanks. sundials boredom who abstracted nope. 


Epoch: 26, Loss: 49.9463
  hint coast bureau—gatsby mediterranean—then police different all—except [ moan. heat dipped eberhardt reasons it—i salon 


Epoch: 31, Loss: 49.9391
  clouds underhand crowd. rumored expression highest villainous punch begged garage. spasms houses time— notice yolks 


Epoch: 36, Loss: 49.9344
  rising decline camp asserted room.the hat-boxes mind.jordan montenegro murmur carraways self-consciously newspapers—a ponies reassuring police 


Epoch: 41, Loss: 49.9047
  chair. series floated big grateful frogs dollars.i the than crepe-de-chine , s how unequally d 


Epoch: 46, Loss: 49.8760
  jay. deauville polisher thought speech started—it waiting porch.he impenetrable inquired. dakota wreaths stalled lyric pleasantly 
